You have to work on the [University dataset](https://drive.google.com/drive/folders/1Hs3nRtK_F3h8eg59B4-TD1DEua6g8Klv?usp=sharing). It contains three different university rankings:
*  The Times Higher Education World University Ranking, shortly *Times*,
*  the Academic Ranking of World Universities, shortly *Shanghai*,
*  the Center for World University Rankings, shortly *cwur*.

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).

https://www.kaggle.com/mylesoneill/world-university-rankings

## 0.Import data

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
url="https://drive.google.com/file/d/1bxVqGhJaNaP9s3ETJsFw4HR7Y7u_Lujc/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
times=pd.read_csv(path,na_values='-',sep=',', thousands=',')
times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,20152.0,8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,11074.0,9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,15596.0,7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,NaN,94.2,7929.0,8.4,27%,45 : 55,2011


In [3]:
times.tail()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2598,601-800,Yeungnam University,South Korea,18.6,24.3,10.9,26.5,35.4,NaN,21958.0,15.3,3%,48 : 52,2016
2599,601-800,Yıldız Technical University,Turkey,14.5,14.9,7.6,19.3,44.0,NaN,31268.0,28.7,2%,36 : 64,2016
2600,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,NaN,4122.0,3.7,3%,NaN,2016
2601,601-800,Yokohama National University,Japan,20.1,23.3,16.0,13.5,40.4,NaN,10117.0,12.1,8%,28 : 72,2016
2602,601-800,Yuan Ze University,Taiwan,16.2,17.7,18.3,28.6,39.8,NaN,8663.0,20.6,4%,43 : 57,2016


In [4]:
times.shape

(2603, 14)

In [5]:
times.columns

Index(['world_rank', 'university_name', 'country', 'teaching', 'international',
       'research', 'citations', 'income', 'total_score', 'num_students',
       'student_staff_ratio', 'international_students', 'female_male_ratio',
       'year'],
      dtype='object')

In [6]:
times.isnull().any() #controllo sui valori mancanti per ogni colonna

world_rank                False
university_name           False
country                   False
teaching                  False
international              True
research                  False
citations                 False
income                     True
total_score                True
num_students               True
student_staff_ratio        True
international_students     True
female_male_ratio          True
year                      False
dtype: bool

In [7]:
# Controllo nelle colonne che mi servono se ci sono dei duplicati
times[times[['world_rank', 'university_name', 'country', 'income', 'num_students', 'female_male_ratio', 'year']].duplicated()]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year


In [8]:
url="https://drive.google.com/file/d/1rhjxG5mP_GlvAlLDdmWnrzDzfnSgAdcf/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
shanghai=pd.read_csv(path)
shanghai.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005


In [9]:
shanghai.shape

(4897, 11)

In [10]:
shanghai.tail()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
4892,401-500,University of Trieste,11-20,NaN,0.0,0.0,5.0,10.9,25.1,20.1,2015
4893,401-500,University of Zaragoza,9-13,NaN,0.0,0.0,7.6,5.1,33.3,13.1,2015
4894,401-500,Utah State University,126-146,NaN,13.6,0.0,3.6,10.8,25.1,15.5,2015
4895,401-500,Vienna University of Technology,4-6,NaN,0.0,0.0,0.0,12.2,28.8,22.9,2015
4896,401-500,Wake Forest University,126-146,NaN,0.0,0.0,14.9,7.5,25.0,11.9,2015


In [11]:
shanghai.isnull().any()

world_rank         False
university_name     True
national_rank       True
total_score         True
alumni              True
award               True
hici                True
ns                  True
pub                 True
pcp                 True
year               False
dtype: bool

In [12]:
shanghai= shanghai[shanghai.university_name.notnull()]

In [13]:
shanghai[shanghai[['university_name', 'world_rank', 'year']].duplicated()]

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
1234,203-304,Queen's University,24-33,NaN,22.0,0.0,0.0,12.8,31.0,21.3,2007
1740,201-302,Queen's University,23-33,NaN,20.9,0.0,0.0,14.4,33.2,25.4,2008
2742,201-300,Queen's University,20-30,NaN,19.9,0.0,0.0,14.8,33.3,16.2,2010


In [14]:
shanghai[shanghai['university_name'] == 'Queen\'s University']

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
164,153-202,Queen's University,6-8,NaN,0.0,0.0,19.2,17.0,35.2,18.5,2005
660,151-200,Queen's University,6-8,NaN,0.0,0.0,18.8,16.5,36.6,20.2,2006
1233,203-304,Queen's University,8-17,NaN,0.0,0.0,16.6,17.1,35.4,19.2,2007
1234,203-304,Queen's University,24-33,NaN,22.0,0.0,0.0,12.8,31.0,21.3,2007
1739,201-302,Queen's University,7-18,NaN,0.0,0.0,16.3,14.8,35.6,27.0,2008
1740,201-302,Queen's University,23-33,NaN,20.9,0.0,0.0,14.4,33.2,25.4,2008
2240,201-302,Queen's University,7-18,NaN,0.0,0.0,16.2,15.2,35.8,20.2,2009
2741,201-300,Queen's University,9-18,NaN,0.0,0.0,16.1,15.3,36.8,20.3,2010
2742,201-300,Queen's University,20-30,NaN,19.9,0.0,0.0,14.8,33.3,16.2,2010
3240,201-300,Queen's University,9-18,NaN,0.0,0.0,16.1,14.1,35.4,19.2,2011


L'università del Queen si ripete perché, durante il 2007, il 2008 e il 2010, il rank nazionale cambia ma 
quello mondiale rimane costante

In [15]:
url="https://drive.google.com/file/d/1KMbNlxJDdSQhT0LeBC0vGMOXUv0jTZGI/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
cwur=pd.read_csv(path)
cwur.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [16]:
cwur.rename({'institution': 'university_name'}, inplace=True, axis= 1)
cwur.columns

Index(['world_rank', 'university_name', 'country', 'national_rank',
       'quality_of_education', 'alumni_employment', 'quality_of_faculty',
       'publications', 'influence', 'citations', 'broad_impact', 'patents',
       'score', 'year'],
      dtype='object')

Institution corrisponde a university name del dataset times, quindi ne modifichiamo il nome

In [17]:
cwur.shape

(2200, 14)

In [18]:
cwur.tail()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
2195,996,University of the Algarve,Portugal,7,367,567,218,926,845,812,969.0,816,44.03,2015
2196,997,Alexandria University,Egypt,4,236,566,218,997,908,645,981.0,871,44.03,2015
2197,998,Federal University of Ceará,Brazil,18,367,549,218,830,823,812,975.0,824,44.03,2015
2198,999,University of A Coruña,Spain,40,367,567,218,886,974,812,975.0,651,44.02,2015
2199,1000,China Pharmaceutical University,China,83,367,567,218,861,991,812,981.0,547,44.02,2015


In [19]:
#controllo valori nulli
cwur.isnull().any()

world_rank              False
university_name         False
country                 False
national_rank           False
quality_of_education    False
alumni_employment       False
quality_of_faculty      False
publications            False
influence               False
citations               False
broad_impact             True
patents                 False
score                   False
year                    False
dtype: bool

In [20]:
cwur[cwur[['university_name', 'country', 'world_rank', 'year']].duplicated()]

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year


### 1. For each university, extract from the *times* dataset the most recent and the least recent data, obtaining two separate dataframes

In [21]:
# Controllo valori mancanti sul nome delle università e dell'anno
len(set(times.university_name)), times[['university_name', 'year']].isnull().any()

(818,
 university_name    False
 year               False
 dtype: bool)

In [22]:
seri=times.groupby(['university_name','year']).size()
seri[seri>1]

Series([], dtype: int64)

Ci siamo assicurati che ci fosse un'unica osservazione per ogni combinazione anno/università. Questo per non rischiare di avere più massimi/minimi riferiti alla stessa università (e dunque eventuali inconsistenze).

In [23]:
massimi_year=times.groupby('university_name').idxmax()['year']
massimi_year.head()

university_name
AGH University of Science and Technology    2405
Aalborg University                          2003
Aalto University                            2056
Aarhus University                           1908
Aberystwyth University                      2105
Name: year, dtype: int64

Per ogni università abbiamo calcolato l'id dell'osservazione che riporta l'anno più recente

In [24]:
most_recent=times.loc[massimi_year]
most_recent

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,NaN,35569.0,17.0,1%,NaN,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,18%,48 : 52,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,2400.0,7.9,20%,46 : 54,2016
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,NaN,2218.0,8.0,14%,49 : 51,2016
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,30%,18 : 82,2016
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,9666.0,10.5,54%,27 : 73,2016


E' stato ripetuto lo stesso procedimento anche per le osservazioni meno recenti

In [25]:
minimi_year=times.groupby('university_name').idxmin()['year']
minimi_year.head()

university_name
AGH University of Science and Technology    2405
Aalborg University                           501
Aalto University                             502
Aarhus University                            166
Aberystwyth University                       476
Name: year, dtype: int64

In [26]:
least_recent=times.loc[minimi_year]
least_recent

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,NaN,35569.0,17.0,1%,NaN,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,NaN,17422.0,15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,NaN,16099.0,24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,23895.0,13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,NaN,9252.0,19.2,18%,48 : 52,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,42,École Normale Supérieure,France,66.8,44.9,48.2,95.7,30.7,68.6,2400.0,7.9,20%,46 : 54,2011
99,100,École Normale Supérieure de Lyon,France,51.1,37.6,34.4,88.8,26.1,57.0,2218.0,8.0,14%,49 : 51,2011
38,39,École Polytechnique,France,57.9,77.9,56.1,91.4,NaN,69.5,2429.0,4.8,30%,18 : 82,2011
47,48,École Polytechnique Fédérale de Lausanne,Switzerland,55.0,100.0,56.1,83.8,38.0,66.5,9666.0,10.5,54%,27 : 73,2011


In [27]:
times.groupby('university_name').size().head()

university_name
AGH University of Science and Technology    1
Aalborg University                          5
Aalto University                            5
Aarhus University                           6
Aberystwyth University                      5
dtype: int64

Si può notare che per le università che compaiono una sola volta nel dataset Times l'osservazione più recente e quella meno recente coincidono. Un esempio è la "AGH University of Science and Technology"

In [28]:
most_recent[most_recent['university_name']=='AGH University of Science and Technology'][['university_name','year']]

,university_name,year
2405,AGH University of Science and Technology,2016


In [29]:
least_recent[least_recent['university_name']=='AGH University of Science and Technology'][['university_name','year']]

,university_name,year
2405,AGH University of Science and Technology,2016


### 2. For each university, compute the improvement in `income` between the least recent and the most recent data points

Notiamo innanzitutto che le colonne 'income' dei due dataset contengono dei valori mancanti:

In [30]:
most_recent.isnull()['income'].sum()

44

In [31]:
least_recent.isnull()['income'].sum()

111

In [32]:
mergiato=pd.merge(most_recent[['university_name','income']],least_recent[['university_name','income']],on='university_name',suffixes=('_mostr', '_leastr'))
mergiato

,university_name,income_mostr,income_leastr
0,AGH University of Science and Technology,NaN,NaN
1,Aalborg University,43.7,36.4
2,Aalto University,61.6,61.9
3,Aarhus University,68.3,61.5
4,Aberystwyth University,31.3,35.5
...,...,...,...
813,École Normale Supérieure,37.1,30.7
814,École Normale Supérieure de Lyon,31.7,26.1
815,École Polytechnique,82.3,NaN
816,École Polytechnique Fédérale de Lausanne,65.4,38.0


In [33]:
mergiato['improvement']=mergiato['income_mostr']-mergiato['income_leastr']
mergiato

,university_name,income_mostr,income_leastr,improvement
0,AGH University of Science and Technology,NaN,NaN,NaN
1,Aalborg University,43.7,36.4,7.3
2,Aalto University,61.6,61.9,-0.3
3,Aarhus University,68.3,61.5,6.8
4,Aberystwyth University,31.3,35.5,-4.2
...,...,...,...,...
813,École Normale Supérieure,37.1,30.7,6.4
814,École Normale Supérieure de Lyon,31.7,26.1,5.6
815,École Polytechnique,82.3,NaN,NaN
816,École Polytechnique Fédérale de Lausanne,65.4,38.0,27.4


In [34]:
mergiato.dropna(inplace=True)
mergiato

,university_name,income_mostr,income_leastr,improvement
1,Aalborg University,43.7,36.4,7.3
2,Aalto University,61.6,61.9,-0.3
3,Aarhus University,68.3,61.5,6.8
4,Aberystwyth University,31.3,35.5,-4.2
5,Adam Mickiewicz University,28.7,28.7,0.0
...,...,...,...,...
812,Zhejiang University,96.2,70.3,25.9
813,École Normale Supérieure,37.1,30.7,6.4
814,École Normale Supérieure de Lyon,31.7,26.1,5.6
816,École Polytechnique Fédérale de Lausanne,65.4,38.0,27.4


### 3. Find the university with the largest increase computed in the previous point

In [35]:
mergiato[mergiato['improvement']==mergiato['improvement'].max()]

,university_name,income_mostr,income_leastr,improvement
428,TU Dresden,99.7,31.9,67.8


Abbiamo usato questo procedimento che, a differenza della idxmax, ci permette di capire se 
esistono più di un'università con il maggiore improvement 

### 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for *Aarhus University* the value is 122-73=49). Notice that some rankings are expressed as a range

Innanzitutto abbiamo ricavato, per ogni ranking, le osservazioni più recenti e abbiamo unito le posizioni delle diverse classifiche in un'unica tabella

In [36]:
most_recent2=shanghai.loc[shanghai.groupby('university_name').idxmax()['year']]
most_recent2.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
4697,301-400,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,2015
4797,401-500,Aalto University,4-6,NaN,0.0,0.0,0.0,10.4,33.4,17.0,2015
4469,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015
4497,101-150,Aix Marseille University,5-6,NaN,13.6,0.0,15.2,20.3,50.1,27.6,2015
3115,102-150,Aix-Marseille University,4-6,NaN,15.6,0.0,16.1,22.4,44.3,15.3,2011


In [37]:
most_recent3=cwur.loc[cwur.groupby('university_name').idxmax()['year']]
most_recent3.head()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
1981,782,AGH University of Science and Technology,Poland,4,267,279,218,708,891,511,896.0,398,44.26,2015
1764,565,Aalborg University,Denmark,5,367,439,218,596,660,287,520.0,401,44.59,2015
1620,421,Aalto University,Finland,4,367,224,218,336,403,645,424.0,246,45.10,2015
1321,122,Aarhus University,Denmark,2,310,165,106,91,81,101,95.0,154,49.65,2015
2013,814,Aberystwyth University,United Kingdom,59,367,567,218,842,552,511,754.0,607,44.22,2015


In [38]:
lista=['university_name','world_rank']
appo=pd.merge(most_recent[lista],most_recent2[lista], on='university_name',suffixes=('_1', '_2'),how='outer')
appo

,university_name,world_rank_1,world_rank_2
0,AGH University of Science and Technology,601-800,NaN
1,Aalborg University,201-250,301-400
2,Aalto University,251-300,401-500
3,Aarhus University,=106,73
4,Aberystwyth University,301-350,NaN
...,...,...,...
1087,West Virginia University,NaN,403-510
1088,Western University,NaN,201-300
1089,Western University (The University of Western ...,NaN,201-300
1090,Xian Jiao Tong University,NaN,201-300


Nell'analisi abbiamo considerato anche le università che non necessariamente compaiono in tutti e tre i ranking. Questo è stato possibile specificando all'interno della funzione 'merge' la modalità 'outer'

In [39]:
riassunto=pd.merge(appo,most_recent3[['university_name','world_rank']], on='university_name',how='outer')
riassunto.rename(columns={"world_rank": "world_rank_3"},inplace=True)
riassunto

,university_name,world_rank_1,world_rank_2,world_rank_3
0,AGH University of Science and Technology,601-800,NaN,782.0
1,Aalborg University,201-250,301-400,565.0
2,Aalto University,251-300,401-500,421.0
3,Aarhus University,=106,73,122.0
4,Aberystwyth University,301-350,NaN,814.0
...,...,...,...,...
1447,École Polytechnique de Montréal,NaN,NaN,781.0
1448,École centrale de Lyon,NaN,NaN,881.0
1449,École normale supérieure - Paris,NaN,NaN,37.0
1450,École normale supérieure de Cachan,NaN,NaN,721.0


Notiamo che, come preannunciato, alcune posizioni sono riportate in forma intervallare, altre con il simbolo '=' davanti e altre ancora correttamente sotto forma di numero. Con la regex seguente abbiamo verificato che queste tre possibilità esaurissero l'insieme dei valori possibili osservabili nelle colonne due e tre. L'ultima colonna non ha particolari problemi ed è già riportata secondo la tipologia corretta (float64).

In [40]:
riassunto['world_rank_1'].str.contains('^=?(\d+)(-\d+)?$',na=False).sum()+riassunto['world_rank_1'].isnull().sum()

c:\users\andrea\appdata\local\programs\python\python36-32\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


1452

In [41]:
riassunto['world_rank_2'].str.contains('^=?(\d+)(-\d+)?$',na=False).sum()+riassunto['world_rank_2'].isnull().sum()

1452

In [42]:
riassunto.isnull().sum()

university_name      0
world_rank_1       634
world_rank_2       794
world_rank_3       428
dtype: int64

In [43]:
print(riassunto['world_rank_1'].dtype)
print(riassunto['world_rank_2'].dtype)
print(riassunto['world_rank_3'].dtype)

object
object
float64


Abbiamo definito la funzione 'sostituzione' che permette di convertire in intero le posizioni specificate in world_rank_1 e world_rank_2. Per quanto riguarda le posizioni espresse in forma intervallare abbiamo deciso di effettuare la media intera tra l'estremo superiore e quello inferiore

In [44]:
def sostituzione(row,colonna):
    if(type(row[colonna]) is str):
        p1=row[colonna].strip('=').split('-')
        if(len(p1)==1):
            return int(p1[0])
        else:
            return (int(p1[0])+int(p1[1]))//2
riassunto['new_world_rank_1']=riassunto.apply(sostituzione,axis=1,args=('world_rank_1',))
riassunto['new_world_rank_2']=riassunto.apply(sostituzione,axis=1,args=('world_rank_2',))
riassunto

,university_name,world_rank_1,world_rank_2,world_rank_3,new_world_rank_1,new_world_rank_2
0,AGH University of Science and Technology,601-800,NaN,782.0,700.0,NaN
1,Aalborg University,201-250,301-400,565.0,225.0,350.0
2,Aalto University,251-300,401-500,421.0,275.0,450.0
3,Aarhus University,=106,73,122.0,106.0,73.0
4,Aberystwyth University,301-350,NaN,814.0,325.0,NaN
...,...,...,...,...,...,...
1447,École Polytechnique de Montréal,NaN,NaN,781.0,NaN,NaN
1448,École centrale de Lyon,NaN,NaN,881.0,NaN,NaN
1449,École normale supérieure - Paris,NaN,NaN,37.0,NaN,NaN
1450,École normale supérieure de Cachan,NaN,NaN,721.0,NaN,NaN


In [45]:
riassunto['differenza_massima']=riassunto.max(axis=1, numeric_only=True, skipna=False)-riassunto.min(axis=1, numeric_only=True,skipna=False)
riassunto

,university_name,world_rank_1,world_rank_2,world_rank_3,new_world_rank_1,new_world_rank_2,differenza_massima
0,AGH University of Science and Technology,601-800,NaN,782.0,700.0,NaN,NaN
1,Aalborg University,201-250,301-400,565.0,225.0,350.0,340.0
2,Aalto University,251-300,401-500,421.0,275.0,450.0,175.0
3,Aarhus University,=106,73,122.0,106.0,73.0,49.0
4,Aberystwyth University,301-350,NaN,814.0,325.0,NaN,NaN
...,...,...,...,...,...,...,...
1447,École Polytechnique de Montréal,NaN,NaN,781.0,NaN,NaN,NaN
1448,École centrale de Lyon,NaN,NaN,881.0,NaN,NaN,NaN
1449,École normale supérieure - Paris,NaN,NaN,37.0,NaN,NaN,NaN
1450,École normale supérieure de Cachan,NaN,NaN,721.0,NaN,NaN,NaN


Forniamo anche la versione che tiene conto solo delle università presenti in tutti e tre i ranking

In [46]:
riassunto.dropna()

,university_name,world_rank_1,world_rank_2,world_rank_3,new_world_rank_1,new_world_rank_2,differenza_massima
1,Aalborg University,201-250,301-400,565.0,225.0,350.0,340.0
2,Aalto University,251-300,401-500,421.0,275.0,450.0,175.0
3,Aarhus University,=106,73,122.0,106.0,73.0,49.0
6,Aix-Marseille University,251-300,102-150,206.0,275.0,126.0,149.0
18,Aristotle University of Thessaloniki,601-800,401-500,459.0,700.0,450.0,250.0
...,...,...,...,...,...,...,...
803,Yale University,12,11,11.0,12.0,11.0,1.0
804,Yeshiva University,=164,201-300,171.0,164.0,250.0,86.0
808,Yonsei University,301-350,201-300,98.0,325.0,250.0,227.0
809,York University,301-350,401-500,337.0,325.0,450.0,125.0


### 5. Consider only the most recent data point of the *times* dataset. Compute the number of male and female students for each country.

In [47]:
most_recent.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,NaN,35569.0,17.0,1%,NaN,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,18%,48 : 52,2016


In [48]:
most_recent.isnull().any()

world_rank                False
university_name           False
country                   False
teaching                  False
international             False
research                  False
citations                 False
income                     True
total_score                True
num_students               True
student_staff_ratio        True
international_students     True
female_male_ratio          True
year                      False
dtype: bool

In [49]:
most_recent['female_male_ratio'].iloc[1]

'48 : 52'

In [50]:
most_recent['perc_femmine']=most_recent['female_male_ratio'].str.extract(r'^(\d+)')
most_recent['perc_maschi']=most_recent['female_male_ratio'].str.extract(r'(\d+)$')
most_recent=most_recent.astype({"perc_femmine": float, "perc_maschi": float})
most_recent['numero_femmine']=(most_recent['num_students']*most_recent['perc_femmine']/100).round()
most_recent['numero_maschi']=(most_recent['num_students']*most_recent['perc_maschi']/100).round()
most_recent

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,perc_femmine,perc_maschi,numero_femmine,numero_maschi
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,NaN,35569.0,17.0,1%,NaN,2016,NaN,NaN,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,15%,48 : 52,2016,48.0,52.0,8363.0,9059.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,17%,32 : 68,2016,32.0,68.0,5152.0,10947.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,54.0,46.0,12903.0,10992.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,18%,48 : 52,2016,48.0,52.0,4441.0,4811.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,2400.0,7.9,20%,46 : 54,2016,46.0,54.0,1104.0,1296.0
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,NaN,2218.0,8.0,14%,49 : 51,2016,49.0,51.0,1087.0,1131.0
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,30%,18 : 82,2016,18.0,82.0,437.0,1992.0
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,9666.0,10.5,54%,27 : 73,2016,27.0,73.0,2610.0,7056.0


If some values in column are missing (NaN) and then converted to numeric, always dtype is float. You cannot convert values to int. Only to float, because type of NaN is float.

Abbiamo controllato che la somma tra numero di maschi e numero di femmine fosse effettivamente uguale al numero di studenti totali:

In [51]:
most_recent[most_recent['num_students']!=most_recent['numero_femmine']+most_recent['numero_maschi']]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,perc_femmine,perc_maschi,numero_femmine,numero_maschi
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,NaN,35569.0,17.0,1%,NaN,2016,NaN,NaN,NaN,NaN
2412,601-800,Amrita University,India,20.4,29.6,5.8,20.7,28.0,NaN,17273.0,6.6,8%,NaN,2016,NaN,NaN,NaN,NaN
2205,401-500,Aston University,Unted Kingdom,18.4,92.5,20.4,51.9,34.6,NaN,NaN,NaN,NaN,NaN,2016,NaN,NaN,NaN,NaN
2421,601-800,Beijing Institute of Technology,China,25.9,16.9,21.7,9.1,87.6,NaN,NaN,NaN,NaN,NaN,2016,NaN,NaN,NaN,NaN
2153,351-400,Bilkent University,Turkey,22.6,47.5,17.5,64.1,30.2,NaN,NaN,NaN,NaN,NaN,2016,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863,=60,Washington University in St Louis,United States of America,54.3,44.9,55.6,99.2,33.5,67.0,12528.0,5.7,17%,NaN,2016,NaN,NaN,NaN,NaN
2301,401-500,Wuhan University,China,38.2,36.7,26.8,18.0,97.8,NaN,46227.0,14.4,7%,NaN,2016,NaN,NaN,NaN,NaN
1814,12,Yale University,United States of America,86.5,64.3,87.8,97.2,43.3,87.4,11751.0,4.4,20%,50 : 50,2016,50.0,50.0,5876.0,5876.0
2600,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,NaN,4122.0,3.7,3%,NaN,2016,NaN,NaN,NaN,NaN


In [52]:
gruppi=most_recent.groupby('country')[['numero_maschi','numero_femmine']].sum()
gruppi

,numero_maschi,numero_femmine
country,,
Argentina,41182.0,67191.0
Australia,321640.0,391736.0
Austria,66113.0,68364.0
Bangladesh,41393.0,21323.0
Belarus,9084.0,20219.0
...,...,...
Unisted States of America,0.0,0.0
United Arab Emirates,4931.0,9516.0
United Kingdom,613028.0,711814.0


In [53]:
gruppi.isnull().any()

numero_maschi     False
numero_femmine    False
dtype: bool

In [54]:
gruppi[(gruppi['numero_maschi']==0)&(gruppi['numero_femmine']==0)]

,numero_maschi,numero_femmine
country,,
Unisted States of America,0.0,0.0
Unted Kingdom,0.0,0.0


In [55]:
most_recent[most_recent['country']=='Unisted States of America']

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,perc_femmine,perc_maschi,numero_femmine,numero_maschi
2019,201-250,University of Hawai’i at Mānoa,Unisted States of America,32.0,63.4,35.5,67.7,44.7,NaN,NaN,NaN,NaN,NaN,2016,NaN,NaN,NaN,NaN


In [56]:
most_recent[most_recent['country']=='Unted Kingdom']

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,perc_femmine,perc_maschi,numero_femmine,numero_maschi
2205,401-500,Aston University,Unted Kingdom,18.4,92.5,20.4,51.9,34.6,NaN,NaN,NaN,NaN,NaN,2016,NaN,NaN,NaN,NaN


Si può notare che due campi country sono stati inseriti in maniera errata

### 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [57]:
most_recent['rapporto']=most_recent['perc_femmine']/most_recent['perc_maschi']

In [58]:
rapportozero=most_recent[most_recent['perc_maschi']==0]
rapportozero.reset_index(inplace=True)
rapportozero

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,perc_femmine,perc_maschi,numero_femmine,numero_maschi,rapporto
0,2227,401-500,Ewha Womans University,South Korea,23.9,34.3,18.3,50.4,57.6,NaN,17625.0,14.4,6%,100 : 0,2016,100.0,0.0,17625.0,0.0,inf


Eliminiamo tutte le scuole che, essendo esclusivamente femminili, hanno un rapporto pari a +Infinito (in questo caso solo una, ovvero la Ewha Womans University)

In [59]:
listar=list(rapportozero['index'])
listar

[2227]

In [60]:
appoggio=most_recent.drop(listar)
appoggio.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,perc_femmine,perc_maschi,numero_femmine,numero_maschi,rapporto
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,NaN,35569.0,17.0,1%,NaN,2016,NaN,NaN,NaN,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,15%,48 : 52,2016,48.0,52.0,8363.0,9059.0,0.923077
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,17%,32 : 68,2016,32.0,68.0,5152.0,10947.0,0.470588
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,54.0,46.0,12903.0,10992.0,1.173913
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,18%,48 : 52,2016,48.0,52.0,4441.0,4811.0,0.923077


In [61]:
media_globale=appoggio['rapporto'].mean()
media_globale

1.081102092089723

In [62]:
punto6=appoggio[appoggio['rapporto']<media_globale]
punto6

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,perc_femmine,perc_maschi,numero_femmine,numero_maschi,rapporto
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,15%,48 : 52,2016,48.0,52.0,8363.0,9059.0,0.923077
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,17%,32 : 68,2016,32.0,68.0,5152.0,10947.0,0.470588
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,18%,48 : 52,2016,48.0,52.0,4441.0,4811.0,0.923077
2406,601-800,Ajou University,South Korea,19.5,20.0,11.9,23.9,45.7,NaN,12706.0,11.3,2%,33 : 67,2016,33.0,67.0,4193.0,8513.0,0.492537
2408,601-800,Alexandria University,Egypt,20.3,33.7,8.2,14.1,29.7,NaN,127431.0,23.3,1%,46 : 54,2016,46.0,54.0,58618.0,68813.0,0.851852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,251-300,Zhejiang University,China,44.3,19.6,46.0,36.1,96.2,NaN,47508.0,15.9,5%,41 : 59,2016,41.0,59.0,19478.0,28030.0,0.694915
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,2400.0,7.9,20%,46 : 54,2016,46.0,54.0,1104.0,1296.0,0.851852
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,NaN,2218.0,8.0,14%,49 : 51,2016,49.0,51.0,1087.0,1131.0,0.960784
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,30%,18 : 82,2016,18.0,82.0,437.0,1992.0,0.219512


### 7. For each country compute the fraction of students that are in one of the universities computed in the previous point.

In [63]:
punto7=punto6.groupby(['country'],as_index=False).sum()[['country','num_students']]
punto7.head()

,country,num_students
0,Australia,146235.0
1,Austria,61033.0
2,Bangladesh,62716.0
3,Brazil,278774.0
4,Canada,86779.0


In [64]:
gfd=most_recent.groupby(['country'],as_index=False).sum()[['country','num_students']]
gfd

,country,num_students
0,Argentina,108373.0
1,Australia,743627.0
2,Austria,134477.0
3,Bangladesh,62716.0
4,Belarus,29303.0
...,...,...
67,Unisted States of America,0.0
68,United Arab Emirates,14447.0
69,United Kingdom,1324842.0
70,United States of America,3280303.0


In [65]:
insiem=pd.merge(punto7,gfd,on='country',how='right')
insiem

,country,num_students_x,num_students_y
0,Argentina,NaN,108373.0
1,Australia,146235.0,743627.0
2,Austria,61033.0,134477.0
3,Bangladesh,62716.0,62716.0
4,Belarus,NaN,29303.0
...,...,...,...
67,Unisted States of America,NaN,0.0
68,United Arab Emirates,5226.0,14447.0
69,United Kingdom,259977.0,1324842.0
70,United States of America,1726340.0,3280303.0


In [66]:
insiem['num_students_x']=insiem['num_students_x'].fillna(0)
insiem

,country,num_students_x,num_students_y
0,Argentina,0.0,108373.0
1,Australia,146235.0,743627.0
2,Austria,61033.0,134477.0
3,Bangladesh,62716.0,62716.0
4,Belarus,0.0,29303.0
...,...,...,...
67,Unisted States of America,0.0,0.0
68,United Arab Emirates,5226.0,14447.0
69,United Kingdom,259977.0,1324842.0
70,United States of America,1726340.0,3280303.0


In [67]:
insiem['frazione']=insiem['num_students_x']/insiem['num_students_y']
insiem

,country,num_students_x,num_students_y,frazione
0,Argentina,0.0,108373.0,0.000000
1,Australia,146235.0,743627.0,0.196651
2,Austria,61033.0,134477.0,0.453855
3,Bangladesh,62716.0,62716.0,1.000000
4,Belarus,0.0,29303.0,0.000000
...,...,...,...,...
67,Unisted States of America,0.0,0.0,NaN
68,United Arab Emirates,5226.0,14447.0,0.361736
69,United Kingdom,259977.0,1324842.0,0.196232
70,United States of America,1726340.0,3280303.0,0.526275


In [68]:
insiem.dropna().head() #escludiamo Unisted States of America e Unted Kingdom

,country,num_students_x,num_students_y,frazione
0,Argentina,0.0,108373.0,0.000000
1,Australia,146235.0,743627.0,0.196651
2,Austria,61033.0,134477.0,0.453855
3,Bangladesh,62716.0,62716.0,1.000000
4,Belarus,0.0,29303.0,0.000000


### 8. Read the file `educational_attainment_supplementary_data.csv`, discarding any row without `country_name` or `series_name`

In [69]:
url="https://drive.google.com/file/d/1VFpCwild-pdSp6I5e7La1ytNWKAaJVGt/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
#path = 'C:/Users/Andrea/OneDrive/Desktop/educational_attainment_supplementary_data.csv'
educational=pd.read_csv(path,na_values='-',sep=',', thousands=',')
educational.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


Abbiamo innanzitutto estratto le osservazioni da eliminare:

In [70]:
educational[(educational['country_name'].isnull())|(educational['series_name'].isnull())]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79054,Last Updated: 10/20/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


E di seguito le abbiamo eliminate:

In [71]:
educational.dropna(axis=0, subset=['country_name','series_name'],inplace=True)
educational

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN


In [72]:
len(educational[(educational['country_name'].isnull())|(educational['series_name'].isnull())])

0

### 9. From `attainment` build a dataframe with the same data, but with 4 columns: `country_name`, `series_name`, `year`, `value`

melt: from wide to long format

In [73]:
educational.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


In [74]:
nuovo=pd.melt(educational,['country_name','series_name'],educational.iloc[:,2:],var_name='year')
nuovo

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54
...,...,...,...,...
2134345,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134346,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134347,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN
2134348,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN


In [75]:
nuovo.dropna()

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33000
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03000
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83000
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34000
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54000
...,...,...,...,...
2054020,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.48356
2054021,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.88820
2054022,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000
2054023,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000


### 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [76]:
d1=pd.DataFrame(times['university_name'].unique(),columns=['university_name'])
d1.head()

,university_name
0,Harvard University
1,California Institute of Technology
2,Massachusetts Institute of Technology
3,Stanford University
4,Princeton University


In [77]:
d2=pd.DataFrame(shanghai['university_name'].unique(),columns=['university_name'])
d2.head()

,university_name
0,Harvard University
1,University of Cambridge
2,Stanford University
3,"University of California, Berkeley"
4,Massachusetts Institute of Technology (MIT)


In [78]:
d3=pd.DataFrame(cwur['university_name'].unique(),columns=['university_name'])
d3.head()

,university_name
0,Harvard University
1,Massachusetts Institute of Technology
2,Stanford University
3,University of Cambridge
4,California Institute of Technology


In [79]:
rewi=pd.concat([d1,d2,d3])
rewi

,university_name
0,Harvard University
1,California Institute of Technology
2,Massachusetts Institute of Technology
3,Stanford University
4,Princeton University
...,...
1019,Shenzhen University
1020,Tianjin Medical University
1021,Babeș-Bolyai University
1022,Henan Normal University


In [80]:
rewi.groupby('university_name',as_index=False).size()

,university_name,size
0,AGH University of Science and Technology,2
1,Aalborg University,3
2,Aalto University,3
3,Aarhus University,3
4,Aberystwyth University,2
...,...,...
1447,École centrale de Lyon,1
1448,École normale supérieure - Paris,1
1449,École normale supérieure de Cachan,1
1450,École normale supérieure de Lyon,1


### 11. In the times ranking, compute the number of times each university appears

In [81]:
counter=times.groupby('university_name').size()
counter

university_name
AGH University of Science and Technology    1
Aalborg University                          5
Aalto University                            5
Aarhus University                           6
Aberystwyth University                      5
                                           ..
École Normale Supérieure                    6
École Normale Supérieure de Lyon            6
École Polytechnique                         6
École Polytechnique Fédérale de Lausanne    6
Örebro University                           1
Length: 818, dtype: int64

In [82]:
punto_11 = pd.DataFrame(counter, columns = ['Appearing']).reset_index()
punto_11

,university_name,Appearing
0,AGH University of Science and Technology,1
1,Aalborg University,5
2,Aalto University,5
3,Aarhus University,6
4,Aberystwyth University,5
...,...,...
813,École Normale Supérieure,6
814,École Normale Supérieure de Lyon,6
815,École Polytechnique,6
816,École Polytechnique Fédérale de Lausanne,6


### 12. Find the universities that appear at most twice in the times ranking.

In [83]:
punto_11[punto_11['Appearing'] <= 2]

,university_name,Appearing
0,AGH University of Science and Technology,1
5,Adam Mickiewicz University,1
6,Aix-Marseille University,1
7,Ajou University,1
9,Alexandru Ioan Cuza University,1
...,...,...
806,Yokohama City University,1
807,Yokohama National University,1
810,Yuan Ze University,2
811,Yıldız Technical University,1


### 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [84]:
dfappo=pd.merge(times[['world_rank','university_name','year']], shanghai[['world_rank','university_name','year']],on=['world_rank','university_name','year'])
dfappo.head()

,world_rank,university_name,year
0,1,Harvard University,2011
1,2,Stanford University,2012
2,9,University of Chicago,2012
3,11,Yale University,2012
4,96,Michigan State University,2012


In [85]:
dfappo=dfappo.astype({'world_rank': 'int64'})

In [86]:
pd.merge(dfappo, cwur[['world_rank','university_name','year']],on=['world_rank','university_name','year'])

,world_rank,university_name,year
0,2,Stanford University,2013
